# Lasso回帰

正則化項にパラメータの絶対値の和を用いた線形回帰モデル。L1正則化とも呼ばれる。Lassoの読み方はラッソ。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

$$
\begin{align}
R(\bm\theta) &= \sum_{i=1}^m |\theta_i| = \|\bm\theta\|_1 \\
J(\bm w)
    &= \| \bm y - \bm X \bm w \|_2^2 + \alpha R(\bm w) \\
    &= \| \bm y - \bm X \bm w \|_2^2 + \alpha \| \bm w \|_1
\end{align}
$$

これを解く。

さて、これまで通り微分して0になる点を求めよう、といきたいところだが、絶対値が含まれているので普通には解けない。絶対値が微分できないので。

解析的に解くことができないので、数値的に解く。Lassoの最適化アルゴリズムは色々あるが、ここでは座標降下法を使って解いてみる。

座標降下法ではパラメータを一つずつ最適化する。パラメータを

$$
\bm w = (w_1,w_2,\cdots,w_m)^T\in\mathbb R^m
$$

とすると、$w_1$, $w_2$, $\cdots$, $w_m$の順に最適化していく。$w_i$の最適化はそれ以外のパラメータ$w_{j\neq i}$を固定して行う。

では$w_i$の最適化について説明する。といっても、目的関数を$w_i$で微分して0になる点を求めるだけ。絶対値が含まれているので微分ができないと言う話であったが、一変数の場合、**劣微分**を導入することで最適解が求められる。

劣微分とは微分の定義を少し拡張したようなもので、関数$f(x)$の$x=a$における劣微分は、「任意の$x$に対して$f(x)\geq f(a)+c(x-a)$が成り立つような$c$の集合」と定義される。

よくわからない人は具体例を見ると良い。例えば、$f(x)=|x|$の$x=0$における劣微分は$[-1,1]$である。

In [2]:
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML

x = np.linspace(-1, 1, 100)
y = np.abs(x)

fig, ax = plt.subplots(figsize=(4, 4))
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.plot(x, y, label="f(x)=|x|")
line, = ax.plot([], [], color="red", label="subgradient", alpha=0.7)
ax.legend()

ani = FuncAnimation(
    fig,
    lambda frame: line.set_data(x, x * frame),
    frames=np.linspace(-1, 1, 25),
    interval=50,
    repeat=False,
)
plt.close()
display(HTML(ani.to_jshtml()))

「$x=0$を通り且つ$f(x)$を超えない直線（劣勾配という。動画の赤い直線）の傾き$c$」が$x=0$における劣微分となる。今回の場合は$-1\sim1$の範囲。

以上を踏まえ、$f(x)=|x|$の微分を以下に定義する。

$$
f'(x) = \begin{cases}
1 & (x>0) \\
[-1,1] & (x=0) \\
-1 & (x<0) \\
\end{cases}
$$

こうすることで、$x=0$における傾きが考えられるようになる。座標降下法では、

In [ ]:
# x = np.linspace(-2, 5, 100)
# y = (5 - x) ** 2 + 5 * np.abs(x)
# plt.plot(x, y)

In [3]:
class LassoRegression:
    def __init__(self, alpha: float = 1., max_iter: int = 1000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.weights = None

    @staticmethod
    def soft_threshold(x, thre):
        if x > thre:
            return x - thre
        elif x < -thre:
            return x + thre
        else:
            return 0

    def fit(self, X, y):
        # X = np.insert(X, 0, 1, axis=1)
        n_features = X.shape[1]
        self.weights = np.zeros(n_features)
        for _ in range(self.max_iter):
            for i in range(n_features):
                weights = self.weights.copy()
                weights[i] = 0
                r = y - X @ weights
                c = X[:, i] @ r
                a = np.linalg.norm(X[:, i]) ** 2
                self.weights[i] = self.soft_threshold(c / a, self.alpha)

    def predict(self, X):
        # X = np.insert(X, 0, 1, axis=1)
        return X @ self.weights